<a href="https://colab.research.google.com/github/phisan-chula/2021-LDP_Design/blob/main/LDP_Inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Design of a Low Distortion Projection for Mini Project**
Phisan Santitamonont, Chulalongkorn University 2022
Phisan.Chula@gmail.com

In [55]:
%%capture
! pip install pygeodesy
! pip install pyproj
! pip install requests
! pip install folium

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# quite slow...
# ! wget   -O tgm2017-1.pgm   'https://www.priabroy.name/?smd_process_download=1&download_id=8512'
#  TGM17 = r'/content/tgm2017-1.pgm'
# download tgm2017 geoid model to you own
TGM17 = r'/content/drive/MyDrive/Geodesy_Cache/tgm2017-1.pgm'

In [58]:
import requests
import pygeodesy as pgd

def dd2DMS( dd, PREC=7, POS=''  ):
    '''conver degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )

POS =[13.5411528, 99.8240430]  # test  lat,lng
###################################################
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
####################################################

GEOID = pgd.geoids.GeoidKarney( TGM17 )
UNDUL = GEOID.height( 14, 100 )
HAE = UNDUL + MSL
print( f'HAE={HAE:.1f} m. ,  N={UNDUL:.1f} m. , MSL={MSL:.1f} m.' )
ELLPS  = pgd.datums.Ellipsoids.WGS84
RG = ELLPS.rocGauss( POS[0] )

HAE=-27.3 m. ,  N=-32.3 m. , MSL=5.0 m.


In [59]:
import pandas as pd
import pyproj
print( f'latitude : {dd2DMS(POS[0]):}  longitude : {dd2DMS(POS[1]):}  ')
k0 = 1 + HAE/RG
TM = r'+proj=tmerc +lat_0=0.0 +lon_0={} +k_0={}  +x_0={}  +y_0={}  +a={} +b={} +units=m +no_defs'
LDP = pyproj.CRS( TM.format( 99+49/60, k0, +2500, -1490000, ELLPS.a, ELLPS.b ) )  # fine tune , serveral times
if 1:
    df = pd.DataFrame( {'Point':['APoint',], 'lat': [POS[0],], 'lng': [POS[1],], 'MSL': [MSL,]  } )
    def CalcCSF( row ):
        h = UNDUL + row.MSL     # FIX UNDUL
        HSF = RG/(RG+h)            # FIX  RG
        FACTOR = pyproj.Proj( LDP).get_factors( row.lng, row.lat )
        PSF = FACTOR.meridional_scale
        CSF = PSF*HSF
        CSF_ppm = (CSF-1)*1E6
        TR = pyproj.Transformer.from_crs( 'epsg:4326', LDP )
        LDP_E,LDP_N = TR.transform( row.lat,row.lng )
        return [h,HSF,PSF,CSF,CSF_ppm, LDP_E, LDP_N]
    df[['h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']] =\
                df.apply( CalcCSF, axis=1, result_type='expand')
print( LDP )

latitude : 13°32′28.15008″  longitude : 99°49′26.5548″  
+proj=tmerc +lat_0=0.0 +lon_0=99.81666666666666 +k_0=0.9999957085539712  +x_0=2500  +y_0=-1490000  +a=6378137.0 +b=6356752.314245179 +units=m +no_defs +type=crs


In [60]:
df

,Point,lat,lng,MSL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,APoint,13.541153,99.824043,5.0,-27.289675,1.000004,0.999996,1.0,0.007927,3298.447013,7573.650079


In [71]:
from io import StringIO
CSV = '''Point,lat,lng,MSL
RTK1,13.542153,99.824043,20.0
RTK2,13.540153,99.821043,-30.0
RTK3,13.541153,99.822043,2.0
'''
dfRTK = pd.read_csv( StringIO(CSV) )
dfRTK[['h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']] = dfRTK.apply( CalcCSF, axis=1, result_type='expand')
dfRTK = dfRTK.round( {'h':1, 'CSF_ppm':1 , 'LDP_E':3, 'LDP_N':3  } )
dfRTK

,Point,lat,lng,MSL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,RTK1,13.542153,99.824043,20.0,-12.3,1.000002,0.999996,0.999998,-2.4,3298.444,7684.307
1,RTK2,13.540153,99.821043,-30.0,-62.3,1.000010,0.999996,1.000006,5.5,2973.716,7463.030
2,RTK3,13.541153,99.822043,2.0,-30.3,1.000005,0.999996,1.000000,0.5,3081.958,7573.667


In [75]:
import folium
icon=folium.Icon(color='red', icon='plus' )
map = folium.Map(location =[POS[0],POS[1]],  zoom_start = 15 )
mkPOS = folium.Marker(location=[POS[0],POS[1]] , icon=icon, popup='POS' )
mkPOS.add_to(map)
for i,row in dfRTK.iterrows():
    #mk = folium.Marker(location=[row.lat, row.lng], popup=row['Point'] )
    folium.CircleMarker(location=[row.lat, row.lng],  radius=10,   
                color='red',  fill_color ='red',  fill_opacity=0.5 ).add_to(map)
map